# Figure making tutorial
Author: Sander Keemink (sander.keemink@donders.ru.nl)

In this notebook we will generate some basic simulation results, and adjust the plotting paremeters.

In [ ]:
%matplotlib inline
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

###  2D systems: coupled neural populations
We will be simulating the following system and trying to plot its behavior in a logical way.

$$ \frac{dr_E}{dt} = -r_E + f(w_{EE}r_E - w_{EI}r_I+I) $$
$$ \frac{dr_I}{dt} = -r_I + f(w_{IE}r_E - w_{II}r_I+I) $$
$$ f(x) = \frac{1}{1+\exp(-sx)} $$
<center><img src='neural systems - exc-inh pair.svg' width=600>
(often referred to as Wilson-Cowan models)

In [ ]:
# solver
def euler_solve(dx, x0, par, dt, nT, noise=None):
    '''Simulates the system of differential equations specified by dx, using forward Euler.
    
    Parameters
    ----------
    dx : function
        Function of form dx(x, par), which determines how the state will change given current state
    x0 : array
        Starting parameters
    par : unknown
        Whatever parametes go into dx
    dt : float
        Timestep size
    nT : int
        Number of timesteps
    noise : array, to be implemented
        If given, what noise to be added at every time step (could possibly be a function)
    
    Output
    ------
    array
        Simulated values
    '''
    # preamble
    nS = len(x0) # nr of equations/signals
    out = np.zeros((nS, nT)) # predefined output
    out[:, 0] = x0
    
    # loop over time
    for i in range(nT-1):
        out[:, i+1] = out[:, i] + dt*dx(out[:, i], par)
    
    # return
    return out

Set parameters

In [ ]:
# timescale
taus = [1, 1]


# connectivity
taus = [0.1, 1]
s = 1
wEE = 5
wIE = 30
wEI = 5
wII = 4.4
W = np.array([[wEE, -wEI], [wIE, -wII]])
Is = [0.3, -10]

# collect parameters
par = [s, W, Is]

# functions
f = lambda x, s: 1/(1+np.exp(-s*x))
drdt = lambda r, t, par: -r/taus + f(np.dot(par[1], r) + par[2], par[0])/taus

Simulate

In [ ]:
# set starting condition
r0 = np.array([0, 0])

# set timepoints to store
times = np.linspace(0, 10, 1000)

# simulate
sol = odeint(drdt, r0, times, args=(par, ))
rE = sol[:, 0]
rI = sol[:, 1]

Plot the activities against each other

In [ ]:
def plotrErI(fontsize=10):
    plt.plot(rE, rI)
    plt.xlabel('Excitatory activity rE (a.u.)', fontsize=fontsize)
    plt.ylabel('Inhibitory activity rI (a.u.)', fontsize=fontsize)
    plt.xlim([0, 1])
    plt.ylim([0, 1])
plotrErI(fontsize=10)

Plot the activities across time

In [ ]:
def plotTimesVSr(r, label, fontsize=10):
    plt.plot(times, r)
    plt.xlabel('Time (a.u.)', fontsize=fontsize)
    plt.ylabel(label, fontsize=fontsize)
plotTimesVSr(rE, 'rE (a.u.)')
plotTimesVSr(rI, 'rI (a.u.)')

Plot all together

In [ ]:
fig = plt.figure()

# plot rE VS rI
ax1 = plt.subplot(121, aspect=2)
plt.xticks(np.linspace(0, 1, 5))
plt.yticks(np.linspace(0, 1, 5))
plotrErI()

# plot rE over time
ax2 = plt.subplot(222, aspect=10)
plotTimesVSr(rE, 'rE')
plt.xticks(np.linspace(0, 10, 5))
plt.yticks(np.linspace(0, 1, 5))

# plot rI over time
ax3 = plt.subplot(224, aspect=10)
plotTimesVSr(rI, 'rI')
plt.xticks(np.linspace(0, 10, 5))
plt.yticks(np.linspace(0, 1, 5))

plt.savefig('fig.svg')
plt.tight_layout() # this often fixes a lot of layout problems. Try plotting without it!